SigMF to MATLAB data converter

This code belongs to:
https://github.com/nasimsoltani/DataFactory/tree/main/SigMF

I made some minor changes and only converts sigmf to matlab.

In [56]:
# enter path of folder where bin and json files are stored
sigmf_path = '/mypath/'

# enter path where matlab files are going to be stored
mat_path =   '/myotherpath/'

In [2]:
from scipy.io import savemat
import glob
import numpy as np
import os
from tqdm import tqdm

In [3]:
device_list = ['m1001','m1005','m1007','m1008','m1009','m10010','m10011']
distance_list = ['6ft','9ft','12ft','15ft']
uav_list = []
for i in range(len(device_list)):
    uav_list.append('uav'+str(i+1))
print (uav_list)

cf_dict = {'m1008_6ft':'2476500000', 'm1008_9ft':'2406500000', 'm1008_12ft':'2406500000', 'm1008_15ft':'2406500000', 'm1009_6ft':'2426500000', 'm1009_9ft':'2426500000', 'm1009_12ft':'2426500000', 'm1009_15ft':'2436500000', 'm1007_6ft':'2476500000', 'm1007_9ft':'2476500000', 'm1007_12ft':'2476500000', 'm1007_15ft':'2476500000', 'm1005_6ft':'2406500000', 'm1005_9ft':'2406500000', 'm1005_12ft':'2406500000', 'm1005_15ft':'2406500000', 'm10011_6ft':'2416500000', 'm10011_9ft':'2416500000', 'm10011_12ft':'2416500000', 'm10011_15ft':'2416500000', 'm1001_6ft':'2406500000', 'm1001_9ft':'2406500000', 'm1001_12ft':'2406500000', 'm1001_15ft':'2406500000', 'm10010_6ft':'2406500000', 'm10010_9ft':'2406500000', 'm10010_12ft':'2406500000', 'm10010_15ft':'2416500000'}

['uav1', 'uav2', 'uav3', 'uav4', 'uav5', 'uav6', 'uav7']


In [87]:
# if destination directory does not exist, create it
if not os.path.isdir(mat_path):
    os.mkdir(mat_path)
    
bin_json_list = glob.glob(sigmf_path + '*')
    
for filepath in tqdm(bin_json_list):
    # do this only if you have a .bin file not a .json file
    if filepath.endswith('.bin'):
        # read the .bin file
        with open (filepath,'rb') as handle:
            iq_seq = np.fromfile(handle, dtype='<f2')    # (f2 => float16)
        n_samples = iq_seq.shape[0]/2

        # separate I and Q
        IQ_data = np.zeros((np.int_(n_samples),2),dtype=np.float16)
        IQ_data[:,0] = iq_seq[range(0, iq_seq.shape[0]-1, 2)]    # load all I-values in dimension 0
        IQ_data[:,1] = iq_seq[range(1, iq_seq.shape[0], 2)]      # ...and Q-values in dimension 1
        # convert to Complex I/Q
        Complex_IQ = IQ_data[:,0] + 1j*IQ_data[:,1]

        # reshape the complex sequence to (1,L)
        Complex_IQ = np.expand_dims(Complex_IQ, axis=0)

        # dump the complex sequence into a .mat file
        filename = filepath.split('/')[-1].strip('.bin')
        this_mat_file_path = os.path.join(mat_path, filename)
        this_mat_file_path = this_mat_file_path + '.mat'
        savemat(this_mat_file_path, {'f_sig':Complex_IQ}) 
     